In [2]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [3]:
%cd slt

/content/slt


In [4]:
!chmod +x data/download.sh
!data/download.sh

--2021-01-02 14:00:13--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  21.5MB/s    in 88s     

2021-01-02 14:01:41 (21.0 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-02 14:01:41--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  21.9MB/s    in 6.6s    

2021-01-02 14:01:48 (18.9 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [6]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 13.5MB/s 
     |████████████████████████████████| 112kB 20.5MB/s 
     |████████████████████████████████| 163kB 20.4MB/s 
     |████████████████████████████████| 399kB 26.5MB/s 
     |████████████████████████████████| 2.3MB 30.4MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 2.4MB 49.9MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 296kB 51.5MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 92kB 11.9MB/s 
     |████████████████████████████████| 13.1MB 57.2MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 20.2MB 80.9MB/s 
     

In [7]:
!mkdir data/PHOENIX2014T 
!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

In [10]:
!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [11]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:48<00:00, 144.92it/s]


In [12]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

list_doc['data']['feature_size'] = new_features.shape[1]
list_doc['model']['model_dir'] = "keypoints3d"
list_doc['training']['recognition_loss_weight'] = 40

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-02 14:16:25,460 	Text Hypothesis :	am tag ist es im süden freundlich im   süden und südosten freundlich sonst ist           es   meist freundlich .
2021-01-02 14:16:25,460 	Text Alignment  :	       I   I  I  S     S          S    S     S   S        S          S     S             S    S     S           
2021-01-02 14:16:25,460 ========================================================================================================================
2021-01-02 14:16:25,460 Logging Sequence: dev/23September_2009_Wednesday_tagesschau-7593
2021-01-02 14:16:25,460 	Gloss Reference :	SONNTAG SCHEINEN ALPEN AUCH SCHAUER MOEGLICH
2021-01-02 14:16:25,460 	Gloss Hypothesis:	******* ******** ***** **** ******* ********
2021-01-02 14:16:25,461 	Gloss Alignment :	D       D        D     D    D       D       
2021-01-02 14:16:25,461 	------------------------------------------------------------------------------------------------------------------

In [14]:
!cp -r ./sign_sample_model "../drive/MyDrive/slt/models/keypoints3d_40" 